<a href="https://colab.research.google.com/github/betty6you/Jing_INFO5731_Spring2020/blob/main/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**



(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#CovidVaccine"](https://twitter.com/hashtag/CovidVaccine) from Twitter. 


In [ ]:
# Write your code here
# I will collect the all the customer reviews of the product 2019 Dell labtop on Amazon
!pip install scrapy
import scrapy


class AmazonReviewsSpider(scrapy.Spider):

  name = "reviews"
  allowed_domains=['amazon.com']
  myBaseUrl="https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/product-reviews/B07N49F51N/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber="
  startURL=[]
  for i in range (1,13):
    startURL.append(myBaseUrl+str(i))
    
  def parse(self, response):
    #Get the Review List
    data = response.css('#cm_cr-review_list')
    
    #Get the Name
    name = data.css('.a-profile-name')
            
    #Get the Review Title
    title = data.css('.review-title')
            
    # Get the Ratings
    star_rating = data.css('.review-rating')
    # Get the users Comments
    comments = data.css('.review-text')
    count = 0
           
    # combining the results
    for review in star_rating:
      yield{
      'Rating': ''.join(review.xpath('.//text()').extract()),
      'Comment': ''.join(comments[count].xpath(".//text()").extract())
         }
      count=count+1

# run file in terminal !scrapy runspider AmazonReviewScraping/AmazonReviewScraping/spiders/DellSpider.py -o reviews.csv    
      

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [10]:
# Write your code here

!pip install pandas
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop=stopwords.words('english')

data=pd.read_csv(r"https://raw.githubusercontent.com/betty6you/Jing_INFO5731_Spring2020/main/Review.csv")
data['comment'] = data['comment'].apply(lambda x: x.replace(r"'", ''))
data['comment'] = data['comment'].apply(lambda x: x.replace(r'"', ''))
data['comment'] = data['comment'].apply(lambda x: x.replace(r'[\n\n\n\n\n\n\n\n\n\n  \n  \n    , \n  ', ''))
data['comment'] = data['comment'].apply(lambda x: x.replace(r'\n, \n  \n]', ''))

data['stopwords']=data['comment'].apply(lambda x:len ([x for x in x.split( ) if x in stop])) #remove stopwords
data['special_char']=data['comment'].apply(lambda x: len([x for x in x.split() if x.startswith(('**','Â','â€','©','€™'))])) #Remove sepcial characters as noise

data['commnet'] = data['comment'].str.rstrip(string.digits)#remove numbers
data['comment']=data['comment'].str.replace('[^\w\s]','') # punctuation removal

data['comment']=data['comment'].apply(lambda x:" ".join(x.lower() for x in x.split()))# lower casing

from nltk.stem import PorterStemmer
st = PorterStemmer()
data['comment'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()])) # Stemming

from textblob import Word
nltk.download('wordnet')
data['comment']= data['comment'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) # Lemmatization

data['comment'].to_csv("AmazonReviewupdated.csv",header=None,index=False) # only 1 column with clean sentences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Question 3

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [79]:
# Write your code here
# POS Tagging
!pip install pandas


import pandas as pd
import nltk
import spacy
nltk.download('averaged_perceptron_tagger')
from collections import Counter
from spacy import displacy


with open('AmazonReviewupdated.csv') as fp:
  x=fp.readlines()
  for a in x:
    b=a.split()
    tags=nltk.pos_tag(b)
    counts = Counter( tag for word,  tag in tags)
    print(counts)
    
    
    displacy.render(nlp(a),jupyter=True)#dependency parsing 
    

    nlp = spacy.load("en_core_web_sm")#Name entity recognition
    doc=nlp(a)
    for ent in doc.ents:
      print(ent.text, ent.start_char, ent.end_char, ent.label_)
 

#Constituency Parsing and Dependency Parsing



# Loading spaCy’s en model and adding benepar model to its pipeline










3 week 127 133 DATE
amazon 155 161 ORG
one month 277 286 DATE
just a few hour 405 420 TIME
amazon 612 618 ORG
amazon 714 720 ORG
Counter({'NN': 5, 'VBD': 2, 'JJ': 2, 'CC': 2, 'DT': 1, 'VBN': 1, 'TO': 1, 'VB': 1, 'RB': 1, 'WP': 1, 'PRP': 1, 'VBP': 1, 'IN': 1, 'VBG': 1})


Counter({'NN': 7, 'IN': 7, 'RB': 4, 'VB': 3, 'CD': 3, 'DT': 2, 'JJ': 2, 'VBN': 2, 'VBZ': 1, 'VBG': 1, 'VBD': 1, 'RP': 1, 'MD': 1, 'TO': 1, 'PRP$': 1})


november 2020 83 96 DATE
july 28 2020 114 126 DATE
Counter({'IN': 2, 'DT': 2, 'NN': 2, 'RB': 2, 'VBD': 1, 'CD': 1, 'PRP$': 1, 'VBN': 1, 'RP': 1, 'PRP': 1, 'VBP': 1, 'JJ': 1})


2 7 8 CARDINAL
Counter({'NN': 14, 'VB': 3, 'DT': 3, 'VBD': 3, 'IN': 3, 'RB': 2, 'VBN': 2, 'MD': 1, 'TO': 1, 'CC': 1, 'JJ': 1})


march 120 125 DATE
january 158 165 DATE
Counter({'JJ': 2, 'NN': 1})


Counter({'NN': 25, 'JJ': 22, 'RB': 11, 'VB': 9, 'CC': 6, 'DT': 6, 'IN': 6, 'VBZ': 5, 'TO': 4, 'VBP': 3, 'PRP': 3, 'MD': 2, 'VBN': 1, 'RP': 1, 'VBD': 1, 'VBG': 1, 'NNS': 1})


trackpad 79 87 ORG
doe 263 266 ORG
touchscreen 275 286 ORG
buzzy 332 337 PERSON
Counter({'NN': 67, 'VBD': 22, 'IN': 21, 'RB': 18, 'JJ': 18, 'DT': 18, 'CC': 14, 'PRP': 13, 'VBG': 10, 'VB': 10, 'VBP': 7, 'CD': 7, 'VBZ': 6, 'VBN': 4, 'TO': 4, 'PRP$': 3, 'WRB': 2, 'NNS': 2, 'RP': 2, 'WDT': 1, 'WP': 1, 'PDT': 1})


3 54 55 CARDINAL
about 15 minute 299 314 TIME
4 minute 479 487 TIME
38 second 488 497 TIME
about 8 minute 613 627 TIME
over 5 year 696 707 DATE
Counter({'NN': 20, 'RB': 13, 'DT': 13, 'JJ': 12, 'IN': 7, 'VB': 6, 'VBD': 4, 'MD': 3, 'PRP': 3, 'VBN': 3, 'VBZ': 2, 'RBR': 2, 'CC': 2, 'VBG': 2, 'VBP': 1, 'TO': 1, 'JJS': 1})


touchscreen 74 85 ORG
asus 372 376 ORG
Counter({'JJ': 2, 'RB': 1, 'IN': 1, 'DT': 1, 'NN': 1})


KeyboardInterrupt: ignored

In [77]:
!pip install benepar
%tensorflow_version 1.x
import benepar
benepar.download('benepar_en3')

from benepar.spacy_plugin import BeneparComponent

# Loading spaCy’s en model and adding benepar model to its pipeline
nlp = spacy.load('en')
nlp.add_pipe(BeneparComponent('benepar_en3'))
with open('AmazonReviewupdated.csv') as fp:
  x=fp.readlines()
  for a in x:
    list(nlp(a).sents[0]._.parse_string)
#text='It took me more than two hours to translate a few pages of English.'

# Generating a parse tree for the text
#list(nlp(text).sents)[0]._.parse_string

[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


TypeError: ignored

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

In [ ]:
'''
Write your explanations of the constituency parsing tree and dependency parsing tree here

A constituency parse tree breaks a text into sub-phrases. Non-terminals in the tree are types of phrases, the terminals are the words in the sentence, and the edges are unlabeled. For a simple sentence "John sees Bill", a constituency parse would be
A dependency parse connects words according to their relationships. Each vertex in the tree represents a word, child nodes are words that are dependent on the parent, and edges are labeled by the relationship. A dependency parse of "John sees Bill", would be
'''

'\nWrite your explanations of the constituency parsing tree and dependency parsing tree here\n\n\n\n'